In [4]:
%load_ext autoreload
%autoreload 2

In [8]:
import os
from datetime import datetime

import pandas as pd
import torch
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torch import nn, optim

from src.dataset import utils
from src.dataset.harvard_skin_cancer import HarvardSkinCancerDatasset
from src.modeling.base_model import BaseModel
from src.modeling.models import DenseNet, Inception, ResNet

In [6]:
df_train = pd.read_csv('artifacts/df_train.csv')
df_valid = pd.read_csv('artifacts/df_valid.csv')
df_test = pd.read_csv('artifacts/df_test.csv')

In [14]:
def experiment(
    model: BaseModel,
    df_train: pd.DataFrame,
    df_valid: pd.DataFrame,
    df_test: pd.DataFrame,
    optimizer,
    criterion,
    n_epochs: int,
) -> None:
    means, stds = utils.extract_mean_and_std_from_images(img_paths=df_train['path'].drop_duplicates())
    transforms = v2.Compose([
        v2.Resize((model.input_size, model.input_size)),
        v2.RandomHorizontalFlip(),
        v2.RandomVerticalFlip(),
        v2.RandomRotation(20),
        v2.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=means, std=stds),
    ])
    
    train_loader = DataLoader(
        dataset=HarvardSkinCancerDatasset(df=df_train, transforms=transforms),
        batch_size=64,
        shuffle=True,
        num_workers=16,
    )
    valid_loader = DataLoader(
        dataset=HarvardSkinCancerDatasset(df=df_valid, transforms=transforms),
        batch_size=64,
        num_workers=16,
    )
    test_loader = DataLoader(
        dataset=HarvardSkinCancerDatasset(df=df_test, transforms=transforms),
        batch_size=64,
        num_workers=16,
    )

    torch.cuda.empty_cache()
    model.train(
        train_loader=train_loader,
        valid_loader=valid_loader,
        criterion=criterion,
        optimizer=optimizer,
        n_epochs=n_epochs,
    )
    model.evaluate(loader=test_loader, criterion=criterion)
    model.save()

In [ ]:
model.evaluate(loader=test_loader, criterion=criterion)

In [16]:
model = Inception(n_classes = df_train['lesion_type_id'].nunique(), extract_features=True)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

experiment(
    model=model,
    optimizer=optim.Adam(model.parameters()),
    criterion=nn.CrossEntropyLoss(),
    df_train=df_train,
    df_valid=df_valid,
    df_test=df_test,
    n_epochs=8,
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8912/8912 [00:20<00:00, 437.99it/s]


[epoch 0], [iter 100 / 613], [train loss 1.00433], [train acc 0.63938]
[epoch 0], [iter 200 / 613], [train loss 0.87346], [train acc 0.68391]
[epoch 0], [iter 300 / 613], [train loss 0.79197], [train acc 0.71406]
[epoch 0], [iter 400 / 613], [train loss 0.74644], [train acc 0.72887]
[epoch 0], [iter 500 / 613], [train loss 0.70206], [train acc 0.74462]
[epoch 0], [iter 600 / 613], [train loss 0.66548], [train acc 0.75745]
MODEL_EVALUATION: acc = 0.8179754273504273 loss = 0.5329369670814939
[epoch 1], [iter 100 / 613], [train loss 0.46839], [train acc 0.83172]
[epoch 1], [iter 200 / 613], [train loss 0.45858], [train acc 0.83359]
[epoch 1], [iter 300 / 613], [train loss 0.44004], [train acc 0.83917]
[epoch 1], [iter 400 / 613], [train loss 0.42873], [train acc 0.84211]
[epoch 1], [iter 500 / 613], [train loss 0.41729], [train acc 0.84556]
[epoch 1], [iter 600 / 613], [train loss 0.40691], [train acc 0.84990]
MODEL_EVALUATION: acc = 0.8358262108262109 loss = 0.5036375158362918
[epoch 2],

In [ ]:
model.device

In [ ]:
image = None

for (images, _) in train_loader:
    image = images
    break

In [ ]:
torch.cuda.empty_cache()
#image = image.to(model.device)
#model.predict(image)
#del model

In [ ]:
for (images, _) in train_loader:
    images = images.to(model.device)
    print(model.predict(images))
    break